In [0]:
from pyspark.sql.functions import *

In [0]:


encounter_hosa = spark.read.parquet("/mnt/bronze/hosa/encounters").withColumn("EncounterID_Key", concat(col("encounterID"),lit("-"),col("datasource")))
display(encounter_hosa)

In [0]:
encounter_hosb = spark.read.parquet("/mnt/bronze/hosb/encounters").withColumn("EncounterID_Key", concat(col("encounterID"),lit("-"),col("datasource")))
display(encounter_hosb)

In [0]:
merged_df = encounter_hosa.unionByName(encounter_hosb)
display(merged_df)

In [0]:
merged_df.createOrReplaceTempView("encounters")

In [0]:
%sql
create or replace temp view quality_checks as (
  select 
                EncounterID as SRC_EncounterID,
                PatientID,
                EncounterDate,
                EncounterType,
                ProviderID,
                DepartmentID,
                ProcedureCode,
                InsertedDate as SRC_InsertedDate,
                ModifiedDate as SRC_ModifiedDate,
                datasource,
                EncounterID_Key,
                Case when EncounterID is null or PatientID is null then true
                else false
                end as is_quarantined
from encounters
)

In [0]:
%sql
select * from quality_checks

In [0]:
%sql
create table if not exists silver.encounters (
                EncounterID string,
                PatientID string,
                EncounterDate date,
                EncounterType string,
                ProviderID string,
                DepartmentID string,
                ProcedureCode integer,
                InsertedDate date,
                ModifiedDate date,
                datasource string,
                EncounterID_Key string,
                is_quarantined boolean,
                audit_InsertedDate timestamp,
                audit_ModifiedDate timestamp,
                is_current boolean
) USING DELTA

In [0]:
%sql
merge into silver.encounters as target 
using quality_checks as source
on target.EncounterID_Key = source.EncounterID_Key
and target.is_current = true
when matched
and (
  target.EncounterID != source.SRC_EncounterID
  or target.PatientID != source.PatientID
  or target.EncounterDate != source.EncounterDate
  or target.EncounterType != source.EncounterType
  or target.ProviderID != source.ProviderID
  or target.DepartmentID != source.DepartmentID
  or target.ProcedureCode != source.ProcedureCode
  or target.audit_InsertedDate != source.SRC_InsertedDate
  or target.audit_ModifiedDate != source.SRC_ModifiedDate
  or target.datasource != source.datasource
  or target.is_quarantined != source.is_quarantined
)then update set
target.audit_ModifiedDate = current_timestamp(),
target.is_current = false
when not matched then insert (
                EncounterID ,
                PatientID ,
                EncounterDate ,
                EncounterType ,
                ProviderID ,
                DepartmentID ,
                ProcedureCode ,
                InsertedDate ,
                ModifiedDate ,
                datasource ,
                EncounterID_Key ,
                is_quarantined ,
                audit_InsertedDate ,
                audit_ModifiedDate ,
                is_current 
)values(
                source.SRC_EncounterID ,
                source.PatientID ,
                source.EncounterDate ,
                source.EncounterType ,
                source.ProviderID ,
                source.DepartmentID ,
                source.ProcedureCode ,
                source.SRC_InsertedDate ,
                source.SRC_ModifiedDate ,
                source.datasource ,
                source.EncounterID_Key ,
                source.is_quarantined ,
                current_timestamp(),
                current_timestamp(),
                true
)

In [0]:
%sql
select  * from silver.encounters

In [0]:

 %sql
  select EncounterID,count(patientid) from  silver.encounters
  group by all
  order by 2 desc